# モデルの出力結果(mongodb)とデータアプリケーション(Nodejs)を連携してみよう

今回はモデルの結果はmongodbに格納済み  
格納したデータをデータアプリケーションであるWebアプリから取得していきます。

「express.js」をもとに話を進めていきましょう。

# モデルの精度をモニタリングしてみましょう

ここまでで、MlOpsの単純な流れは一度みたことになります。

# モデルの再学習を検討しよう

# モデルを継続的にデプロイしよう